In [1]:
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")

In [39]:
dailyPowerConsumptionPath =  "DailyPowerConsumption.txt"
housesPath =  "Houses.txt"
dailyPowerConsumptionRDD = sc.textFile(dailyPowerConsumptionPath).cache()
housesRDD = sc.textFile(housesPath).cache()
output1 = "outPart1" 
output2 = "outPart2"

In [40]:
# Part1
# House
# House102,Turin,Italy,120
# DailyPowerConsumption
# House102,2022/12/21,12
highConsumption = 30.0

def mapHighConsumption(elem):
    if(elem[1] < highConsumption):
        return (elem[0], (1,0))
    else:
        return (elem[0], (0,1))

dailyPowerConsumptions22 = dailyPowerConsumptionRDD\
                        .filter(lambda l: str(l.split(',')[1]).startswith("2022/"))\
                        .map(lambda l: (l.split(',')[0], (float(l.split(',')[2]), 1)))\
                        .reduceByKey(lambda t1, t2: (t1[0]+t2[0], t1[1]+t2[1]))\
                        .map(lambda elem: (elem[0], elem[1][0]/elem[1][1]))\
                        .map(mapHighConsumption)

In [41]:
pairHouseCountry = housesRDD\
            .map(lambda l: (l.split(',')[0], l.split(',')[2]))\
            .join(dailyPowerConsumptions22)\
            .map(lambda i: i[1])

In [42]:
res1 = pairHouseCountry\
    .reduceByKey(lambda t1, t2: (t1[0]+t2[0], t1[1]+t2[1]))\
    .filter(lambda elem: elem[1][1] == 0)\
    .keys()

In [43]:
# res1.saveAsTextFile(output1)
res1.collect()

['France']

In [44]:
# PART2

In [65]:
annualHighConsumption = 10000

dailyPowerConsumptions21 = dailyPowerConsumptionRDD\
                        .filter(lambda l: str(l.split(',')[1]).startswith("2021/"))\
                        .map(lambda l: (l.split(',')[0], float(l.split(',')[2])))\
                        .reduceByKey(lambda v1, v2: v1 + v2)\
                        .filter(lambda elem: elem[1] > annualHighConsumption)\
                        .map(lambda elem: (elem[0], 1))

In [67]:
dailyPowerConsumptions21.collect()

[('House1', 1), ('House3', 1), ('House1183', 1)]

In [84]:
pairHouseCityCountry = housesRDD\
            .map(lambda l: (l.split(',')[0], (l.split(',')[1], l.split(',')[2])))\
            .join(dailyPowerConsumptions21)\
            .map(lambda i: (i[1][0][0],(i[1][0][1], i[1][1])))\
            .reduceByKey(lambda t1, t2: (t1[0], t1[1]+t1[1]))\
            .filter(lambda i: i[1][1] > 500)
            .map

In [85]:
pairHouseCityCountry.collect()

[]